In [1]:
import ta
import ccxt
import time
import pprint
import pandas as pd
import SecretConfig
from IPython.display import clear_output

# Définir l'échange
exchange = ccxt.bitget({
    "apiKey":SecretConfig.APIKey,
    "secret":SecretConfig.SecretKey,
    'password': SecretConfig.Passphrase,
    "options": {'defaultType': 'swap', 'adjustForTimeDifference' : True},
    "timeout": 60000,
    "enableRateLimit": True,
})

In [2]:
# Initialiser un DataFrame vide pour stocker les données roulantes
full_rolling_dataframe = pd.DataFrame()
df_top_10 = pd.DataFrame()

# Définir une fonction pour récupérer et traiter les données
def fetch_data():
    global full_rolling_dataframe
    global df_top_10

    while True:
        # Récupérer les données des contrats à terme
        futures_tickers = exchange.fetch_tickers(params={'type': 'swap'})
        tickers_name_list = futures_tickers.keys()
        # tickers_name_list = []

        '''for ticker in tickers_name_list_brut:
            ticker_name = ticker.split('/')[0]
            tickers_name_list.append(ticker_name)'''

        # Initialiser un dictionnaire pour stocker les nouvelles données
        data = {}

        # Parcourir chaque contrat à terme
        for ticker in tickers_name_list:

            # Récupérer les informations pertinentes pour chaque contrat à terme
            ticker_name = ticker.split('/')[0]
            data[ticker_name + ' Last Price'] = futures_tickers.get(ticker, {}).get('info', {}).get('last')
            data[ticker_name + ' EMA_1'] = None
            data[ticker_name + ' Var EMA_1'] = None
            data[ticker_name + ' EMA_2'] = None
            data[ticker_name + ' Diff Carre EMA_1_2']  = None
            data[ticker_name + ' Divergence EMA_1_2'] = None
            data[ticker_name + ' Divergence Moyen Wind1'] = None
            


        # Convertir le dictionnaire en DataFrame
        data['Timestamp'] = futures_tickers.get(ticker, {}).get('timestamp')
        new_data_df = pd.DataFrame([data])

        # Convertir les colonnes en numérique et définir 'Timestamp' comme index
        new_data_df = new_data_df.apply(pd.to_numeric)
        new_data_df['Timestamp'] = pd.to_datetime(new_data_df['Timestamp'], unit='ms')
        new_data_df['Timestamp'] = new_data_df['Timestamp'].dt.round('10L')
        new_data_df = new_data_df.set_index('Timestamp')

        # Ajouter les nouvelles données au DataFrame roulant
        full_rolling_dataframe = pd.concat([full_rolling_dataframe, new_data_df])

        wind1 = 2
        wind2 = 12
        wind3 = 5

        # Calculer les indicateurs techniques
        for ticker in tickers_name_list:
            ticker_name = ticker.split('/')[0]
            full_rolling_dataframe[ticker_name + ' EMA_1'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window= wind1)
            if len(full_rolling_dataframe) >= wind3:
                full_rolling_dataframe[ticker_name + ' Var EMA_1'] = ((full_rolling_dataframe[ticker_name + ' EMA_1'] - full_rolling_dataframe[ticker_name + ' EMA_1'].iloc[- wind3] ) 
                                                                      / full_rolling_dataframe[ticker_name + ' EMA_1'].iloc[- wind3]) * 100
            else:
                full_rolling_dataframe[ticker_name + ' Var EMA_1']  = None
            full_rolling_dataframe[ticker_name + ' EMA_2'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Last Price'], window= wind2)
            full_rolling_dataframe[ticker_name + ' Diff Carre EMA_1_2'] = (full_rolling_dataframe[ticker_name + ' EMA_1'] - full_rolling_dataframe[ticker_name + ' EMA_2'])**2
            
            
            full_rolling_dataframe[ticker_name + ' Divergence EMA_1_2']= full_rolling_dataframe[ticker_name + ' Diff Carre EMA_1_2'].pct_change(fill_method=None)
            full_rolling_dataframe[ticker_name + ' Divergence Moyen Wind1'] = ta.trend.ema_indicator(full_rolling_dataframe[ticker_name + ' Divergence EMA_1_2'], window= wind1)
            full_rolling_dataframe = full_rolling_dataframe.copy()

        # Convertir en données numériques
        full_rolling_dataframe = full_rolling_dataframe.apply(pd.to_numeric)

        # Limiter le DataFrame roulant aux 600 dernières périodes
        full_rolling_dataframe = full_rolling_dataframe.tail(50)

        # Obtenez la liste des ticker_names
        ticker_names = set(col.split(' ')[0] for col in full_rolling_dataframe.columns if ' Var EMA_1' in col)

        # Sélectionnez les colonnes de variation
        cols_variation = [ticker + ' Var EMA_1' for ticker in ticker_names if full_rolling_dataframe[ticker + ' Divergence Moyen Wind1'].iloc[-1] > 0]

        # Obtenez la dernière ligne du DataFrame
        last_row = full_rolling_dataframe.iloc[-1]

        # Créez une série avec seulement les colonnes de variation de la dernière ligne
        last_row_variation = last_row[cols_variation]

        # Obtenez les 10 plus grandes variations
        top_10_variations = last_row_variation.nlargest(10).keys().tolist()

        top_10_variations = [name.replace(' Var EMA_1', '') for name in top_10_variations]

        # top_10_variations = [f"{name.replace(' Var EMA_1', '')}: {full_rolling_dataframe[name].iloc[-1]}, {full_rolling_dataframe[name.replace(' Var EMA_1', ' Divergence Moyen Wind1')].iloc[-1]}" for name in top_10_variations]

        top_10_variations += [None] * (10 - len(top_10_variations))

        # Créez un nouveau DataFrame avec seulement les 10 plus grandes variations
        '''if df_top_10.empty:
            df_top_10 = pd.DataFrame(top_10_variations)
        else:
            df_top_10 = pd.concat([df_top_10, top_10_variations])'''

        # Créez un nouveau DataFrame avec seulement les 10 plus grandes variations
        if df_top_10.empty:
            df_top_10 = pd.DataFrame([top_10_variations], columns=['top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10'], index=[last_row.name])
        else:
            new_row = pd.DataFrame([top_10_variations], columns=['top1', 'top2', 'top3', 'top4', 'top5', 'top6', 'top7', 'top8', 'top9', 'top10'], index=[last_row.name])
            df_top_10 = pd.concat([df_top_10, new_row])


        df_top_10 = df_top_10.tail(10)
        series = df_top_10.stack()
        counts = series.value_counts()


        # Afficher les 10 dernières lignes du DataFrame
        clear_output(wait=True)
        display(full_rolling_dataframe.tail(20))
        display(df_top_10)
        display(counts)
        display(len(counts))
        # print(df_top_10)
        # pprint.pprint(full_rolling_dataframe[ticker_name + ' ATR'])
        # pprint.pprint(full_rolling_dataframe[ticker_name + ' ATR Change'])
        # pprint.pprint(full_rolling_dataframe)

        # Attendre 1 seconde avant de récupérer à nouveau les données
        time.sleep(0.5)



fetch_data()

,BTC Last Price,BTC EMA_1,BTC Var EMA_1,BTC EMA_2,BTC Diff Carre EMA_1_2,BTC Divergence EMA_1_2,BTC Divergence Moyen Wind1,ETH Last Price,ETH EMA_1,ETH Var EMA_1,...,TWT Diff Carre EMA_1_2,TWT Divergence EMA_1_2,TWT Divergence Moyen Wind1,MEME Last Price,MEME EMA_1,MEME Var EMA_1,MEME EMA_2,MEME Diff Carre EMA_1_2,MEME Divergence EMA_1_2,MEME Divergence Moyen Wind1
Timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-13 20:59:57.500,36780.0,NaN,NaN,NaN,NaN,NaN,NaN,2086.87,NaN,NaN,...,NaN,NaN,NaN,0.034557,NaN,NaN,NaN,NaN,NaN,NaN
2023-11-13 20:59:59.290,36780.0,36780.000000,NaN,NaN,NaN,NaN,NaN,2086.87,2086.870000,NaN,...,NaN,NaN,NaN,0.034609,0.034592,NaN,NaN,NaN,NaN,NaN
2023-11-13 21:00:00.960,36790.0,36786.666667,NaN,NaN,NaN,NaN,NaN,2086.67,2086.736667,NaN,...,NaN,NaN,NaN,0.034609,0.034603,NaN,NaN,NaN,NaN,NaN
2023-11-13 21:00:02.770,36799.4,36795.155556,NaN,NaN,NaN,NaN,NaN,2086.67,2086.692222,NaN,...,NaN,NaN,NaN,0.034614,0.034610,NaN,NaN,NaN,NaN,NaN


,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
2023-11-13 20:59:57.500,None,None,None,None,None,None,None,None,None,None
2023-11-13 20:59:59.290,None,None,None,None,None,None,None,None,None,None
2023-11-13 21:00:00.960,None,None,None,None,None,None,None,None,None,None
2023-11-13 21:00:02.770,None,None,None,None,None,None,None,None,None,None


Series([], Name: count, dtype: int64)

0

KeyboardInterrupt: 